### import

In [1]:
import binascii
import pickle
import os
from os.path import join
import pandas as pd
from tqdm import tqdm
import re

def file_listFnc(path) :
    # 업데이트시 dupl.py의 fileInfo()도 업데이트 해야
    p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif|tiff|m4a|wav|mp[34])$', re.I) 
    return [f.name for f in os.scandir(path) if f.is_file() and (p_extension.search(f.name))]

def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

def read_cks(path_kcs) :
    with open(path_kcs, "rb") as pkl :
        return pickle.load(pkl)

def delete_kcs(filePath, dict_kcs) :
    k = os.path.split(filePath)[1][:8]
    c = str(crc32_checksum(filePath))
    s = str(os.path.getsize(filePath))
    kcs = k + c + s
    if kcs in dict_kcs :
        del dict_kcs[kcs]
        print("삭제하였습니다.")
    else :
      print(filePath, kcs, "없는 키값")

def write_kcs(path_kcs, dict_kcs) :
    with open(path_kcs, "wb") as pkl :
        pickle.dump(dict_kcs, pkl)

def get_sc(fullPath : str) :

    size = str(os.path.getsize(fullPath))
    crc32 = str(crc32_checksum(fullPath))
    sc = size + crc32

    return sc
def get_cs(fullPath : str) :

    size = str(os.path.getsize(fullPath))
    crc32 = str(crc32_checksum(fullPath))
    cs = crc32 + size

    return cs
    #temp = {"sc":sc, "score" : score, "root" : root, "stem":stem, "ext":ext, "fullPath" : fullPath, "size":size, "crc32": crc32, "mtime":mtime, "sell":sell}

### 파일 검색(구스캔파일)

In [4]:
# 파일 불러오기
pathOfDF = r"D:\1.전산,개발\3.파일서버관리\파일\중복조사"
nameOfDF = "sc_fullPath.pkl"
df = pd.read_pickle(join(pathOfDF, nameOfDF)) # dict도 pd.read_pickle로 읽어도 됨
pd.set_option('display.max_colwidth', None)

#### 경로 검색

In [6]:
###############################
searchWord = r"이성익" 
result = df[df.fullPath.str.contains(searchWord)].drop_duplicates(["sc"])
###############################
# 출력물에만 적용되는 설정인가보네. 바로 출력까지 해준다.
result.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

,sc,fullPath
250450,208726EBFC35A1,"D:\구 스캔파일\광주 공유폴더\■집행문,판결문\승계완료, 판결문\이성익 19타채103528.pdf"
250455,27270451403221,"D:\구 스캔파일\광주 공유폴더\■집행문,판결문\승계완료, 판결문\이성익 가소.pdf"
250460,43328BB9B5525,"D:\구 스캔파일\광주 공유폴더\■집행문,판결문\승계완료, 판결문\이성익 사용증명.pdf"
307320,2681512FBCB2FF,D:\구 스캔파일\광주 공유폴더\■초본--작업보류\초본\이성익 (2).pdf
307322,2482194DEE662A,D:\구 스캔파일\광주 공유폴더\■초본--작업보류\초본\이성익.pdf
313705,24517521EB6BD0,D:\구 스캔파일\광주 공유폴더\■초본--작업보류\초본\12월12일\이성익.pdf
318077,55388510C5CF5,D:\구 스캔파일\광주 공유폴더\■초본--작업보류\초본\7월12일 초본\이성익.pdf
372343,729995FE9D8156,D:\구 스캔파일\광주 공유폴더\원인서류 채무자키\20416987 이성익 원인서류.pdf
1542925,6082404D69230,D:\구 스캔파일\광주 공유폴더\통합파일\(1.DNP_헬프)\3.채권양도통지서\20416987 이성익_양통.pdf
1593401,49212EE6E3B3F,D:\구 스캔파일\광주 공유폴더\모음\정리할것\중복확인필요\중복건확인필요\티와이머니1차 양통\이성익 (2).tif


#### sc로 검색

In [17]:
# pathOfDict = r"D:\0.전산\1.진행중과업\파일서버개편\파일\중복조사"
# nameOfDict = "정렬된 sc별 파일정보.pkl"
# dict = pd.read_pickle(join(pathOfDict, nameOfDict))
# # with open(join(pathOfDict, nameOfDict), "rb") as pkl :
# #     dict = pickle.load(pkl)

In [5]:
#######################################################
path = r"\\192.168.0.75\스캔파일\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\1.매입전 자료\추가\양도통지서\금융-01\10690280"
file = "양통2차.pdf"
#######################################################
sc = get_sc(join(path, file))
print("sc : ", sc)
df[df.sc.str.match(sc)]

sc :  50620F7A388BC


,sc,fullPath
2378278,50620F7A388BC,"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\1.매입전 자료\양도통지서(양통) ㅡ신복 채권이라 일부 없다고함\금융-01\10690280\양통2차.pdf"


In [5]:
p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif|tiff|m4a|wav|mp[34])$', re.I) 
path = r"D:\구 스캔파일\서울우담추가\추가"
new_files = []
for root, dirs, files in tqdm(os.walk(path)) : 
    for f in files: 
        if p_extension.search(f) :
            sc = get_sc(join(root, f))
            if len(df[df.sc.str.match(sc)])>0 : pass
            else : 
                new_files.append([root, f])

result = pd.DataFrame(data=new_files, columns=['dir', 'filename'])
result.to_excel(r"D:\구 스캔파일\서울우담추가\추가\new_files.xlsx")

11933it [2:12:25,  1.50it/s]


#### 검색한 sc로 서버에서 찾기

In [5]:
# 구 데이터 열기
with open (r"D:\1.전산,개발\3.파일서버관리\파일\중복조사\검색용서버cs.pkl", "rb") as pkl :
    dict_server = pickle.load(pkl)
len(dict_server)

323030

In [7]:
sc = "6082404D69230" ###############
cs = sc[-8:]+sc[:-8]
try :
    print(dict_server[cs])
except :
    print("동일한 파일이 서버에 없음")

/volume1/솔림헬프/2.양도통지서/DNP-01/20416987/20416987_이성익_양도통지서.pdf


### 파일검색(서버)

In [2]:
# 신규데이터로 cs : path .pkl 만들기 (업데이트 꼭 할 필요는 없음)
# key를 제거하므로 개수는 원본과 다름
pathOfDict = r"\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사"
nameOfDict = "kcs별 파일정보_솔림헬프.pkl"
with open(join(pathOfDict, nameOfDict), "rb") as pkl :
    temp = pickle.load(pkl)
cs = [x[8:] for x in temp.keys()]
n_dict = {}
for k, v in zip(cs, temp.values()) :
    if k not in n_dict :
        n_dict[k] = [v]
    else :
        n_dict[k].append(v)

with open (r"D:\1.전산,개발\3.파일서버관리\파일\중복조사\검색용서버cs.pkl", "wb") as pkl :
    pickle.dump(n_dict, pkl)

In [2]:
# 구 데이터 열기
with open (r"D:\1.전산,개발\3.파일서버관리\파일\중복조사\검색용서버cs.pkl", "rb") as pkl :
    dict_server = pickle.load(pkl)
len(dict_server)

341869

##### 단일 파일 확인

In [2]:
#######################################################
path = r"D:\구 스캔파일\광주 공유폴더\■집행문,판결문\승계완료, 판결문\서울-광주이관 승계집행문--변환가능"
file = "1501001118 이재수 12차전2077.pdf"
#######################################################
cs = get_cs(join(path, file))
try :
    print("cs : ", cs[:8],cs[8:])
    print(dict_server[cs])
except :
    print("동일한 파일이 서버에 없음")

cs :  D763A82C 3179284
동일한 파일이 서버에 없음


##### 파일검색(서버) - 폴더 전체 확인

In [4]:
# 폴더 하나
path = r"C:\Users\SL\Desktop\광주 2_파일명변경완료_ 잘못된 파일들\전산없음"
file_list = file_listFnc(path)
cs_list = []
dupl_list = []
serverPath_list =[]
for f in tqdm(file_list, total = len(file_list)) : 
    fullPath = join(path, f)
    
    crc32 = str(crc32_checksum(fullPath))
    size = str(os.path.getsize(fullPath))
    cs = crc32 + size 
    cs_list.append(cs)
    #서버에 있음
    if cs in dict_server : 
        serverPath_list.append(dict_server[cs])
        dupl_list.append(True)
    else :
        serverPath_list.append("서버 없음")
        dupl_list.append(False)

df_server = pd.DataFrame({"file" : file_list, "dupl" : dupl_list, "cs" : cs_list, "serverPath" : serverPath_list})
print(df_server["dupl"].value_counts())

df_server.to_excel(join(path, "isInServer.xlsx"))

100%|██████████| 18/18 [00:00<00:00, 89.53it/s] 

False    18
Name: dupl, dtype: int64


In [4]:
# 폴더 여러개
path = r"D:\구 스캔파일\대전"
#file_list = file_listFnc(path)
cs_list = []
dupl_list = []
serverPath_list =[]
file_list = []
for root, __dirs__, files in tqdm(os.walk(path)) : 
    for f in files:
        fullPath = join(root, f)
        file_list.append(fullPath)
        
        crc32 = str(crc32_checksum(fullPath))
        size = str(os.path.getsize(fullPath))
        cs = crc32 + size 
        cs_list.append(cs)
        #서버에 있음
        if cs in dict_server.keys() : 
            serverPath_list.append(dict_server[cs])
            dupl_list.append(True)
        else :
            serverPath_list.append("서버 없음")
            dupl_list.append(False)

df_server = pd.DataFrame({"file" : file_list, "dupl" : dupl_list, "cs" : cs_list, "serverPath" : serverPath_list})
print(df_server["dupl"].value_counts())

df_server.to_excel(r"c:/Users/sl/Desktop/isInServer.xlsx")

5it [00:00, 29.33it/s]


False    71
Name: dupl, dtype: int64


In [5]:
df_server[df_server.serverPath != "서버 없음"]

,file,dupl,cs,serverPath


##### cs 단일단순비교

In [14]:
path1 = r"\\192.168.0.75\솔림헬프\1.원인서류\한울가람\20412849\20412849_㈜대영유니콘_원인서류_(1).pdf"
path2 = r"D:\삭제예정파일\서버 중복\1702-1 대영유니콘 (2).pdf"
cs1 = get_cs(path1)
cs2 = get_cs(path2)
print(cs1[:8], cs1[8:])
print(cs2[:8], cs2[8:])

5162AA90 12484977
DE36051F 12505288


### 원래 있던 거

In [2]:
path_kcs = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사/kcs별 파일정보_솔림헬프.pkl'
dict_kcs = read_cks(path_kcs)
print(len(dict_kcs))

244983


In [7]:
path = r"D:\전산부재"
for f in os.listdir(path) :
    filePath = os.path.join(path, f)
    if os.path.isfile(filePath) :
        delete_kcs(filePath, dict_kcs)

print(len(dict_kcs))

D:\전산부재\20411984_노민_집행권원_2021차전109606_서창배.pdf 20411984825CD9ED126021 없는 키값
D:\전산부재\20414770 강상균 5110141.tif 2041477071A9E29016970 없는 키값
244983


In [8]:
write_kcs(path_kcs, dict_kcs)

In [80]:
########## 파일명변경완료 잘못된 거 바로 잡기
path_kcs = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사/kcs별 파일정보_솔림헬프.pkl'
dict_kcs = read_cks(path_kcs)
print("kcs", len(dict_kcs))
cs_list = [k[8:] for k in dict_kcs.keys()]

kcs 245097


In [81]:
from tqdm import tqdm
import pandas as pd
excel_path = r'\\192.168.0.75\스캔파일\스캔파일log\success\구 스캔파일 log\20221005 170358_1638_광주원인서류.csv'
df = pd.read_csv(excel_path, header=None)
df.head()

,0,1,2,3
0,/volume1/스캔파일/새 스캔파일(업로드)/별도작업,20408094 김봉찬 원인서류1.pdf,20408094_김봉찬_원인서류.pdf,/volume1/솔림헬프/1.원인서류/DNP-02/20408094
1,/volume1/스캔파일/새 스캔파일(업로드)/별도작업,20408094 김봉찬 원인서류3.pdf,20408094_김봉찬_원인서류_(1).pdf,/volume1/솔림헬프/1.원인서류/DNP-02/20408094
2,/volume1/스캔파일/새 스캔파일(업로드)/별도작업,20408096 권기연 원인서류1.pdf,20408096_권기연_원인서류.pdf,/volume1/솔림헬프/1.원인서류/DNP-01/20408096
3,/volume1/스캔파일/새 스캔파일(업로드)/별도작업,20408096 권기연 원인서류3.pdf,20408096_권기연_원인서류_(1).pdf,/volume1/솔림헬프/1.원인서류/DNP-01/20408096
4,/volume1/스캔파일/새 스캔파일(업로드)/별도작업,20408098 김옥경 원인서류1.pdf,20408098_김옥경_원인서류.pdf,/volume1/솔림헬프/1.원인서류/DNP-01/20408098


In [82]:
files = df[2].values
roots = df[3].values

In [84]:
import re
data = pd.DataFrame(None, columns=["key", "fullPath", "cs", "cs_count"])
index = 0
for r, f in tqdm(zip(roots, files), total=len(roots)) :
    key = f[:8]
    re_path = re.sub("/volume1", r"\\\\192.168.0.75", r)
    re_path = re.sub("/", r"\\", re_path)
    fullPath = os.path.join(re_path, f)
    print(fullPath)
    if os.path.exists(fullPath) :
        cs = str(crc32_checksum(fullPath)) + str(os.path.getsize(fullPath))
        cs_count = cs_list.count(cs)
        data.loc[len(data)] = [key, fullPath, cs, cs_count]
data.head()


\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408094\20408094_김봉찬_원인서류.pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408094\20408094_김봉찬_원인서류_(1).pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20408096\20408096_권기연_원인서류.pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20408096\20408096_권기연_원인서류_(1).pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20408098\20408098_김옥경_원인서류.pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20408098\20408098_김옥경_원인서류_(1).pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20408099\20408099_이병화_원인서류.pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20408099\20408099_이병화_원인서류_(1).pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408104\20408104_김상학_원인서류.pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408104\20408104_김상학_원인서류_(1).pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408124\20408124_차성희_원인서류.pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408124\20408124_차성희_원인서류_(1).pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408152\20408152_김지형_원인서류.pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408152\20408152_김지형_원인서류_(1).pdf
\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408154\20408154_김현철_원인서류.p

,key,fullPath,cs,cs_count
0,20408094,\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408094\204...,A2AA144E2190957,1
1,20408098,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20408098\204...,5512892C661920,1
2,20408099,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20408099\204...,E5FA1E0B690049,1
3,20408104,\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408104\204...,F8709DCD672524,1
4,20408124,\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20408124\204...,CC049F721198980,1


In [85]:
data.to_excel(r"C:\Users\SL\Desktop\광주 2_파일명변경완료_ 잘못된 파일들\list.xlsx", encoding='utf-8-sig')

In [88]:
delList = data[data.cs_count > 1].fullPath

In [96]:
delList1 = delList.values
delList1

array(['\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-02\\20408157\\20408157_노민정_원인서류_(1).pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-01\\20411217\\20411217_신우진_원인서류.pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-01\\20411218\\20411218_조용석_원인서류_(1).pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-01\\20411222\\20411222_박치윤_원인서류.pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-02\\20411225\\20411225_김계환_원인서류_(1).pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-01\\20411228\\20411228_송영록_원인서류.pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-01\\20411229\\20411229_현필성_원인서류_(1).pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-01\\20411235\\20411235_진승종합물류㈜_원인서류_(1).pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-01\\20411238\\20411238_㈜세븐텔레콤_원인서류.pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-02\\20411243\\20411243_박성열_원인서류_(1).pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-02\\20411244\\20411244_태화이 엔 이㈜_원인서류.pdf',
       '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-02\\20411246\\2041

In [100]:
# 올바른 채무자키로 파일이 업로드 되어 있는 경우
import shutil
import traceback
dst_root = r"C:\Users\SL\Desktop\광주 2_파일명변경완료_ 잘못된 파일들"
result = []
for f in tqdm(delList1, total=len(delList1)) :
    name = os.path.split(f)[1]
    dst = os.path.join(dst_root, name)
    try :
        shutil.move(f, dst)
    except Exception as e:
        print("===================================")
        print(traceback.format_exc())
        continue # 반복문 계속 돌아
    #result.append([f, dst])


  0%|          | 0/646 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "c:\Users\sl\anaconda3\lib\shutil.py", line 823, in move
    os.rename(src, real_dst)
FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-02\\20408157\\20408157_노민정_원인서류_(1).pdf' -> 'C:\\Users\\SL\\Desktop\\광주 2_파일명변경완료_ 잘못된 파일들\\20408157_노민정_원인서류_(1).pdf'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SL\AppData\Local\Temp\ipykernel_13036\1044811990.py", line 9, in <cell line: 5>
    shutil.move(f, dst)
  File "c:\Users\sl\anaconda3\lib\shutil.py", line 843, in move
    copy_function(src, real_dst)
  File "c:\Users\sl\anaconda3\lib\shutil.py", line 444, in copy2
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "c:\Users\sl\anaconda3\lib\shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundError: [Errno 2] No such file or directory: '\\\\192.168.0.75\\솔림헬프\\1.원인서류\\DNP-02\\20408157\\20408157_노민정

100%|██████████| 646/646 [06:47<00:00,  1.58it/s]


In [101]:
remain = data[data.cs_count == 1]
remain.to_excel(r"C:\Users\SL\Desktop\광주 2_파일명변경완료_ 잘못된 파일들\remain.xlsx", encoding='utf-8-sig')

In [104]:
df1 = pd.read_excel(r"C:\Users\SL\Desktop\광주 2_파일명변경완료_ 잘못된 파일들\remain.xlsx", index_col=0)
df1

,key,fullPath,cs,cs_count,rightName
1505,20407790,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20407790\204...,30CEFD7B833678,1,NaN
1506,20407790,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20407790\204...,F41F29781806612,1,20458514
30,20411239,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20411239\204...,D9F93F1A4547622,1,NaN
32,20411239,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20411239\204...,43706686471171,1,20455211
33,20411242,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20411242\204...,5C9050CE9023196,1,NaN
...,...,...,...,...,...
1495,20417779,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20417779\204...,D93F08845233277,1,NaN
1497,20418776,\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20418776\204...,54E5749C324383,1,NaN
1499,20418790,\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20418790\204...,D4B32715613195,1,NaN
1502,20418883,\\192.168.0.75\솔림헬프\1.원인서류\DNP-01\20418883\204...,356BD4DE1572150,1,NaN


In [110]:
# 전산없음
dst_root = r"C:\Users\SL\Desktop\광주 2_파일명변경완료_ 잘못된 파일들\전산없음"
전산없음 = df1[df1.rightName == "전산없음"].fullPath.values
for f in tqdm(전산없음, total=len(전산없음)) :
    name = os.path.split(f)[1]
    dst = os.path.join(dst_root, name)
    try :
        shutil.move(f, dst)
    except Exception as e:
        print("===================================")
        print(traceback.format_exc())
        continue # 반복문 계속 돌아

100%|██████████| 18/18 [00:12<00:00,  1.44it/s]


In [158]:
# 재업로드(수작업으로 올바른 채무자키 찾은 것들 종결건들임..)
dst_root = r"C:\Users\SL\Desktop\광주 2_파일명변경완료_ 잘못된 파일들\재업로드"
df2 = df1.dropna()
reup = df2[df2.rightName != "전산없음"]
for index, values in tqdm(reup.iterrows(), total=len(reup)) :
    src = values.fullPath
    ext = os.path.splitext(src)[1]
    name = str(values.rightName) + " 원인서류" + ext
    dst = os.path.join(dst_root, name)
    #print(src, dst)
    try :
        shutil.move(src, dst)
    except Exception as e:
        print("===================================")
        print(traceback.format_exc())
        continue # 반복문 계속 돌아

100%|██████████| 66/66 [00:35<00:00,  1.87it/s]


In [155]:
os.path.splitext("\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20414875\20414875_오석주_원인서류.pdf")[1]

'.pdf'

In [2]:
path_excel = r"D:\구 스캔파일\서울우담추가\추가\new_files_1.xlsx"
df = pd.read_excel(path_excel)
df

,Unnamed: 0,dir,filename,sn,채무자키,docu,extra
0,34,D:\구 스캔파일\서울우담추가\추가\양도통지서\아폴-01\10632618,엘씨3차_828박기남.pdf,10632618,20459921,양도통지서,아폴
1,35,D:\구 스캔파일\서울우담추가\추가\양도통지서\아폴-01\10632698,엘씨3차_870박장국.pdf,10632698,20460765,양도통지서,아폴
2,36,D:\구 스캔파일\서울우담추가\추가\양도통지서\아폴-01\10632744,박해일730809.pdf,10632744,20462065,양도통지서,아폴
3,37,D:\구 스캔파일\서울우담추가\추가\양도통지서\아폴-01\10632755,엘씨3차_902박홍분.pdf,10632755,20462320,양도통지서,아폴
4,38,D:\구 스캔파일\서울우담추가\추가\양도통지서\아폴-01\10632757,엘씨3차_904박희성.pdf,10632757,20464505,양도통지서,아폴
...,...,...,...,...,...,...,...
304,343,D:\구 스캔파일\서울우담추가\추가\원인서류\에캐-01\10650583,1897-최홍석.tif,10650583,20462582,원인서류,NaN
305,344,D:\구 스캔파일\서울우담추가\추가\원인서류\한투-01\10628047,신용회복_159_차미림(약정서).pdf,10628047,20465046,원인서류,NaN
306,345,D:\구 스캔파일\서울우담추가\추가\원인서류\한투-01\10628048,신용회복_160_최낙연(약정서).pdf,10628048,20465341,원인서류,NaN
307,346,D:\구 스캔파일\서울우담추가\추가\원인서류\한투-01\10628054,신용회복_166_윤자영(약정서).pdf,10628054,20466050,원인서류,NaN
